<center>

*******************************************************************************************
    
### GENERATE MUSIC
### WITH SYNTHESISED BELL SOUNDS
  
##### 8 May 2024

##### Juan Ignacio Mendoza Garay  

*******************************************************************************************

</center>

#### INFORMATION:


* Description:

    See title.

* Instructions:

    Edit the values indicated with an arrow like this: <---  
    Comment/uncomment or change values as suggested by the comments.  
    Run the program, close your eyes and hope for the best.  

*******************************************************************************************


In [8]:
import random
import math
import struct
import subprocess
from datetime import datetime

In [ ]:
save_folder = r'C:\Users\Dr_David_Bowman\Documents' # <--- full path to save the resulting mp3 file (prepend r)
output_fn_pfx = 'bells'     # <--- name prefix for the mp3 file (the remainder is a POSIX timestamp)
total_duration = 10         # <--- total duration in seconds
note_classes = [2,4,7,9,11] # <--- note classes to choose from (e.g., 1 = C, 2 = C#, 3 = D, etc.)
octaves = [3,4,5]           # <--- octaves to choose from (e.g., Middle-C and A440 are in octave 4)
note_duration_range = [1,3] # <--- minimum and maximum note duration in seconds (the last note's duration will be the sum of minimum and maximum)
A4 = 440                    # <--- tuning

# Specify where the ffmpeg codec executable is:
ffmpeg_path = r'C:\Users\Dr_David_Bowman\AppData\Local\Programs\Python\Python3xx' # <--- full path to ffmpeg.exe (prepend r)

# Get ffmpeg.exe from here:
# https://github.com/GyanD/codexffmpeg/releases/tag/2022-06-06-git-73302aa193

In [ ]:
def bytes_to_int(b1, b2):
    return struct.unpack('<h', bytes([b1, b2]))[0]

def int_to_bytes(int):
    return struct.pack('<h', int)

def generate_bell_sound(freq, note_duration, amplitude=1.0, sample_rate=44100):
    """
    Generate a single bell sound as a bytearray.
    """
    num_samples = int(sample_rate * note_duration)
    audio_data = bytearray()
    for n in range(num_samples):
        sin_params = 2 * math.pi * (n / sample_rate) * freq
        value_f0 = math.sin( sin_params / 2     ) * amplitude * 0.5
        value_f1 = math.sin( sin_params         ) * amplitude * 0.6
        value_f2 = math.sin( sin_params * 1.183 ) * amplitude * 0.015
        value_f3 = math.sin( sin_params * 1.506 ) * amplitude * 0.01
        value_f4 = math.sin( sin_params * 2     ) * amplitude * 0.1
        value_all = ( value_f0 + value_f1 + value_f2 + value_f3 + value_f4 ) / 1.225
        audio_data += struct.pack('<h', int(value_all * 32767))  # '<h' indicates little-endian 16-bit signed integers
    
    # linear attack:
    attack_time = 0.01 # seconds
    attack_n_samples = int(sample_rate * attack_time)
    if attack_n_samples%2: # ensure it is an even number to comply with bytearray format (relevant for power decay)
        attack_n_samples-=1
    for i in range(0,attack_n_samples,2):
        int_val = bytes_to_int(audio_data[i], audio_data[i + 1])
        tapered_val = int(int_val * i / attack_n_samples)
        audio_data[i], audio_data[i + 1] = int_to_bytes(tapered_val)
    
    # power decay:
    test = 0
    decay_step = (math.exp(1)-1)/(num_samples - attack_n_samples)
    decay_cumm = 0
    for i in range(attack_n_samples,len(audio_data),2):
        int_val = bytes_to_int(audio_data[i], audio_data[i + 1])        
        log_curve_val = math.log(decay_cumm+1)
        if log_curve_val < 1:
            tapered_val = int(int_val * (1-log_curve_val))
        else:
            tapered_val = 1
        decay_cumm += decay_step
        audio_data[i], audio_data[i + 1] = int_to_bytes(tapered_val)
    return audio_data

def encode_mp3(raw_audio_data, full_output_fn, ffmpeg_path, sample_rate=44100):
    """
    Encode and save audio data as mp3.
    """
    process = subprocess.Popen(
        [ffmpeg_path+'\\ffmpeg.exe', '-hide_banner', '-loglevel', 'error', '-y',
         '-f', 's16le',  # signed 16-bit little endian format
         '-ar', str(sample_rate),  # sample rate
         '-ac', '1',  # audio channels (1 for mono)
         '-i', 'pipe:0',  # input from stdin pipe
         full_output_fn],  # output file
        stdin=subprocess.PIPE)
    process.communicate(input=raw_audio_data)
    if process.returncode != 0:
        raise Exception("ffmpeg encoding process failed")

def generate_music_mp3(note_classes,octaves,A4,total_duration,full_output_fn, ffmpeg_path,sample_rate=44100):
    """
    Generate and save music.
    """
    silence_time = 0.3 # silence at the beginning (in seconds)
    current_duration = silence_time
    music_audio_data = bytearray()
    last_note_duration = note_duration_range[0] + note_duration_range[1]

    while current_duration < total_duration:
        
        # Randomly choose a note given note classes and octaves:
        chosen_note_class = random.choice(note_classes)
        chosen_octave = random.choice(octaves)
        chosen_note = chosen_note_class + (chosen_octave-1) * 12 + 21 
        freq = (A4 / 32) * (2 ** ((chosen_note - 9) / 12))
        
        # Random duration within given range, ensuring not to exceed total duration:
        remaining_time = total_duration - current_duration
        if remaining_time <= last_note_duration:
            note_duration = remaining_time
        else:
            note_duration = random.uniform(note_duration_range[0],note_duration_range[1])
        
        music_audio_data += generate_bell_sound(freq, note_duration, amplitude=0.95, sample_rate=sample_rate)
        current_duration += note_duration

    # Add silence at the beginning:
    full_audio_data = bytearray(int(silence_time*2*sample_rate))+music_audio_data
    
    # Encode and save:
    encode_mp3(full_audio_data, full_output_fn, ffmpeg_path, sample_rate)
    print(f"Bell music saved to {full_output_fn}")

if __name__ == "__main__":
    timestamp = str(datetime.now().timestamp()).replace(".", "_")
    full_output_fn = save_folder+'\\'+output_fn_pfx+'_'+timestamp+'.mp3'
    generate_music_mp3(note_classes,octaves,A4,total_duration,full_output_fn,ffmpeg_path)
    